In [1]:
import requests
import json
import numpy as np
import pandas as pd
from copy import deepcopy
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
from Preprocessing import Preprocessor
from constants import RUSSIAN_TIME_ZONES
from Preprocessing import vincenty_dist

mypath = '/mnt/HARD/MinMax94/data/data_all/CSV/Raw_extended/'

## Station_config

In [2]:
stations_mm94 = pd.read_csv('/mnt/HARD/MinMax94/data/CSV/stations_mm94_def.csv')
stations_rp5 = pd.read_csv('/mnt/HARD/MinMax94/data/CSV/stations_rp5_def.csv')

In [3]:
stations = deepcopy(stations_mm94)
stations['station_type'] = "road"
stations['station_nature'] = "real"
stations['timezone'] = stations['timezone'].apply(lambda row: RUSSIAN_TIME_ZONES[row])
stations['road_category'] = 2
#stations['roadlayers'] = str({"1": {"type": "asphalt", "thickness": 0.2}})
stations = stations[['station_id', 'timezone', 'longitude', 'latitude', 
              'station_type', 'station_nature', 'road_category']]

In [4]:
mm94_station_id = 1821
exact_station = stations[stations['station_id']==mm94_station_id]
station_config = exact_station.to_dict(orient='records')[0]
station_config['roadlayers'] = {"1": {"type": "asphalt", "thickness": 0.05},
                                "2": {"type": "asphalt", "thickness": 0.25},
                                "3": {"type": "asphalt", "thickness": 0.20},
                                "4": {"type": "asphalt", "thickness": 0.20}}
#with open("file", "w") as file:
#    json.dump(exact_station_data, file)
    
#with open("file", "r") as file:
#    station_config = json.load(file)

station_config

{'latitude': 34.068667,
 'longitude': 64.02075,
 'road_category': 2,
 'roadlayers': {'1': {'thickness': 0.05, 'type': 'asphalt'},
  '2': {'thickness': 0.25, 'type': 'asphalt'},
  '3': {'thickness': 0.2, 'type': 'asphalt'},
  '4': {'thickness': 0.2, 'type': 'asphalt'}},
 'station_id': 1821,
 'station_nature': 'real',
 'station_type': 'road',
 'timezone': 'MSK'}

## Road_config

In [5]:
road_config = {
    "maintainability_level": "high",
    "category": "2"}

## Global_forecast

In [6]:
## find nearest wmo station
station_data = stations_mm94[stations_mm94['station_id']==mm94_station_id]
mm94_coords = station_data[['latitude', 'longitude']].values[0]

nearest_rp5_station_id = stations_rp5.loc[[stations_rp5.apply(
                                lambda x: vincenty_dist(mm94_coords, (x['latitude'], x['longitude'])), 
                                axis=1).idxmin()]]['station_id'].values[0]

In [9]:
wmo_station_id = nearest_rp5_station_id
mypath = '/mnt/HARD/MinMax94/data/CSV/RP5/'
preprocessor = Preprocessor()

rp5 = pd.read_csv(mypath + str(wmo_station_id) + '.csv', sep=';', skiprows=6, index_col=False,
                 dtype={'VV': str, 'RRR': str})
date_time_col = [col for col in rp5.columns if col.startswith('Местное время')][0]
rp5 = rp5.rename(columns={date_time_col: 'Местное время'})
rp5['station_id'] = wmo_station_id

mmx_wmo_data = preprocessor.ConvertData(rp5, from_format="RP5", to_format="Mmx")
del mmx_wmo_data['data_p_weather'], mmx_wmo_data['data_precip_interval']
mmx_wmo_data = preprocessor.AddUTC(mmx_wmo_data, '/mnt/HARD/MinMax94/data/CSV/stations_rp5_def.csv')
mmx_wmo_interpolated = preprocessor.InterpolatePatterns(mmx_wmo_data)

forecast_data = preprocessor.ConvertData(mmx_wmo_interpolated, from_format="Mmx", to_format="Metro")
forecast_data = forecast_data.set_index('date_time_utc')
forecast_data["p_weather"] = np.nan
del forecast_data['station_id'], forecast_data['wind_gusts']
forecast_data = forecast_data.fillna(0)
forecast_data['p_weather'] = forecast_data['p_weather'].astype(int)
forecast_data['cloudiness'] = forecast_data['cloudiness'].astype(int)

AttributeError: 'Preprocessor' object has no attribute 'ConvertData'

In [8]:
current_date_time = pd.Timestamp(2015, 12, 30, 13, 0)
predict_future_delta = pd.Timedelta(45, unit='h')
predict_previous_delta = pd.Timedelta(3, unit='h')
global_forecast = deepcopy(forecast_data[(forecast_data.index >= pd.Timestamp(2015, 12, 30, 10, 0)) & \
                                (forecast_data.index <= pd.Timestamp(2016, 1, 1, 14, 0))])
global_forecast = global_forecast.set_index('date_time_metro', drop=True)
global_forecast_json = global_forecast.to_dict(orient='index')

## RWIS data

In [10]:
mypath = '/mnt/HARD/MinMax94/data/data_all/CSV/Raw_extended/'

# reading loaded csv files from data_csv directory, output is a list (length=number of stations) of raw df
raw = pd.read_csv(mypath + str(mm94_station_id) + '_raw.csv', parse_dates = ['date_time']) 
raw = raw.reset_index(drop=True)

raw_data = preprocessor.SelectFeatures(raw)
raw_data = preprocessor.PivotTable(raw_data)
mmx_data = preprocessor.ConvertData(raw_data, from_format="Raw", to_format="Mmx")
mmx_data = preprocessor.AddUTC(mmx_data)
mmx_patterns = preprocessor.CreatePatternList(mmx_data, max_gap = pd.Timedelta('4h'))
mmx_interpolated = preprocessor.InterpolatePatterns(mmx_patterns)

rwis_df = preprocessor.ConvertData(mmx_interpolated, from_format="Mmx", to_format="Metro")
rwis_df = rwis_df.set_index('date_time_utc')
rwis_df['salinity'] = 0
rwis_df['freezing_point'] = 0
del rwis_df['cloudiness'], rwis_df['station_id']

In [11]:
rwis_previous_delta = pd.Timedelta(12, unit='h')
rwis_data = rwis_df[(rwis_df.index >= pd.Timestamp(2015, 12, 30, 7, 0)) & \
                                (rwis_df.index <= pd.Timestamp(2015, 12, 30, 13, 0))]
rwis_data = rwis_data.set_index('date_time_metro', drop=True)
rwis_data_json = rwis_data.to_dict(orient='index')

## All together

In [12]:
import simplejson
test_path = '/home/ndsviriden/metro_test.txt'
working_path = '/home/ndsviriden/data_metro_1.txt'

data_for_metro = {"station_config": station_config, 
                  "road_config": road_config,
                  "global_forecast": global_forecast_json,
                  "rwis_data": rwis_data_json}

In [21]:
type(data_for_metro["global_forecast"]['2015-12-30 10:00 UTC']['cloudiness'])

float

In [16]:
data_for_metro["global_forecast"]

{'2015-12-30 10:00 UTC': {'cloudiness': 50.0,
  'humidity': 34.0,
  'p_weather': 0.0,
  'precipitation_intensity': 0.0,
  'precipitation_type': 0.0,
  'pressure': 762.8,
  't_air': 16.8,
  't_dew_point': 0.8,
  'visibility': 4000.0,
  'wind_direction': 247.5,
  'wind_speed': 1.7},
 '2015-12-30 11:00 UTC': {'cloudiness': 60.0,
  'humidity': 35.0,
  'p_weather': 0.0,
  'precipitation_intensity': 0.0,
  'precipitation_type': 0.0,
  'pressure': 761.9,
  't_air': 17.1,
  't_dew_point': 1.5,
  'visibility': 4000.0,
  'wind_direction': 270.0,
  'wind_speed': 1.3},
 '2015-12-30 13:00 UTC': {'cloudiness': 80.0,
  'humidity': 42.7,
  'p_weather': 0.0,
  'precipitation_intensity': 0.0,
  'precipitation_type': 0.0,
  'pressure': 761.5,
  't_air': 15.0,
  't_dew_point': 2.0,
  'visibility': 4000.0,
  'wind_direction': 210.0,
  'wind_speed': 1.0},
 '2015-12-30 14:00 UTC': {'cloudiness': 80.0,
  'humidity': 49.3,
  'p_weather': 0.0,
  'precipitation_intensity': 0.0,
  'precipitation_type': 0.0,
  'pr

In [13]:
#------   
with open(working_path, "r") as file:
    working_data = json.load(file)

In [14]:
with open(test_path, "w") as file:
    simplejson.dump(data_for_metro, file, indent=4, ignore_nan=True)
    
with open(test_path, "r") as file:
    data_for_metro = json.load(file)

In [154]:
k1 = data_for_metro['global_forecast']['2015-12-30 10:00 UTC'].keys()
k2 = working_data['global_forecast']['2015-12-30 10:00 UTC'].keys()

In [32]:
new = '/home/ndsviriden/just_six.txt'
r = {"value": [{"1": 1, "2": 2}, {"3": 3}]}

with open(new, "w") as file:
    simplejson.dump(r, file, indent=4, ignore_nan=True)

In [159]:
data_for_metro['global_forecast']['2015-12-30 10:00 UTC']

{'cloudiness': 51.7,
 'humidity': 34.0,
 'p_weather': 0.0,
 'precipitation_intensity': 0.0,
 'precipitation_type': 0.0,
 'pressure': 762.8,
 't_air': 16.8,
 't_dew_point': 0.8,
 'visibility': 4000.0,
 'wind_direction': 247.5,
 'wind_speed': 1.7}